In [5]:
from langchain_community.document_loaders import BSHTMLLoader, DirectoryLoader

class HTMLDirectoryLoader:
    def __init__(self, directory):
        self.directory = directory

    def load(self):
        loader = DirectoryLoader(path=self.directory, glob="**/*.html", loader_cls=BSHTMLLoader)
        documents = loader.load()
        return documents

In [6]:
import os

input_dir = os.path.join(os.getcwd(), 'data')
db_dir = os.path.join(os.getcwd(), 'database')
loader = HTMLDirectoryLoader(input_dir)
documents = loader.load()

len(documents)

17

In [7]:
from langchain_community.embeddings.ollama import OllamaEmbeddings

embedding_func = OllamaEmbeddings(show_progress=True, model='mistral')
embedding_func

OllamaEmbeddings(base_url='http://localhost:11434', model='mistral', embed_instruction='passage: ', query_instruction='query: ', mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None, show_progress=True, headers=None, model_kwargs=None)

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=20000,
    chunk_overlap=200,
    length_function=len,
)

chunks = text_splitter.split_documents(documents)

chunks[1].to_json()

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'schema', 'document', 'Document'],
 'kwargs': {'metadata': {'source': '/home/metapercept/ChatbotTest/data/use_lazy_loading.html',
   'title': 'Use Lazy Loading'},
  'page_content': "Use Lazy LoadingIntroductionUse Angular CLIMaintain proper folder structureFollow consistent Angular coding stylesUse ES6 featuresUse trackBy along with ngForBreak down into small reusable componentsUse Lazy LoadingUse Index.tsAvoid logic in templates Cache API callsUse async pipe in templatesDeclare safe stringsAvoid any type when declaring constants and variablesState managementUse CDK Virtual ScrollUse Lazy LoadingTry to lazy load the modules in an Angular application whenever possible. Lazy loading loads something only when it is used. This reduces the size of the application load initial time and improves the application boot time by not loading unused modules.Without lazy loading// app.routing.tsimport { WithoutLazyLoadedComponent } from './withou

In [9]:
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings
import ollama

chroma_client = chromadb.PersistentClient(
    path='chroma',
    settings=Settings(),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)

collection = chroma_client.create_collection(name='docs')

idx = [str(i) for i in range(len(documents))]
documents = [chunk.page_content for chunk in chunks]
metadatas = [chunk.metadata for chunk in chunks]
embedding_list = [ollama.embeddings(model='mistral', prompt=chunk.page_content) for chunk in chunks]

collection.add(
    documents = documents,
    embeddings = embedding_list,
    metadatas = metadatas,
    ids = idx
)
collection

# for i, d in enumerate(chunks):  
#     response = ollama.embeddings(model='mistral', prompt=d.page_content)
#     embedding = response['embedding']
#     collection.add(
#         ids=[str(i)],
#         embeddings=[embedding],
#         documents=[d.page_content]
#     )

ValueError: Expected each embedding in the embeddings to be a list, got ['dict']

In [5]:
col1 = chroma_client.get_collection('Testing123')
col1

Collection(id=525cf353-f7c9-48d8-ac1f-11a509a62f53, name=Testing123)

In [6]:
from langchain.prompts import PromptTemplate

PROMPT_TEMPLATE = """
Answer the question based on the context below. If you can't answer the question, reply "I don't know".
Ensure your responses are clear, concise, and helpful.

Context: {context}

Question: {question}

"""

pt = PromptTemplate(
            template=PROMPT_TEMPLATE, input_variables=["context", "question"]
    )

pt

PromptTemplate(input_variables=['context', 'question'], template='\nAnswer the question based on the context below. If you can\'t answer the question, reply "I don\'t know".\nEnsure your responses are clear, concise, and helpful.\n\nContext: {context}\n\nQuestion: {question}\n\n')

In [8]:
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.memory import ConversationSummaryMemory
from langchain_ollama.llms import OllamaLLM
from langchain_community.vectorstores import Chroma

model = OllamaLLM(model='mistral')

database = Chroma(embedding_function=embedding_func, client=chroma_client, collection_name='Testing123')

database

rag = RetrievalQA.from_chain_type(
            llm=model,
            retriever=database.as_retriever(),
            memory=ConversationSummaryMemory(llm = model),
            chain_type_kwargs={"prompt": pt, "verbose": True},
            chain_type='stuff'
            
        )

rag.invoke("What are features of angular?")

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.17it/s]




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Answer the question based on the context below. If you can't answer the question, reply "I don't know".
Ensure your responses are clear, concise, and helpful.

Context: 

Question: What are features of angular?




KeyboardInterrupt: 